# Matching pipline

The comparison method is used in statistical analysis to eliminate distortions caused by differences in the basic characteristics of the studied groups. Simply put, matching helps to make sure that the results of the experiment are really caused by the studied effect, and not by external factors.

Matching is most often performed in cases where the use of a standard AB test is impossible.

In [1]:
import copy

from hypex.dataset import Dataset, InfoRole, TreatmentRole, TargetRole 
from hypex import Matching

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data preparation 

It is important to mark the data fields by assigning the appropriate roles:

* FeatureRole: a role for columns that contain features or predictor variables. Our split will be based on them. Applied by default if the role is not specified for the column.
* TreatmentRole: a role for columns that show the treatment or intervention.
* TargetRole: a role for columns that show the target or outcome variable.
* InfoRole: a role for columns that contain information about the data, such as user IDs.

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int), 
        "post_spends": TargetRole(float)
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.roles

{'user_id': Info(<class 'int'>),
 'treat': Treatment(<class 'int'>),
 'post_spends': Target(<class 'float'>),
 'signup_month': Feature(<class 'int'>),
 'pre_spends': Feature(<class 'float'>),
 'age': Feature(<class 'float'>),
 'gender': Feature(<class 'str'>),
 'industry': Feature(<class 'str'>)}

## Simple Matching  
Now matching has 4 steps: 
1. Dummy Encoder 
2. Process Mahalanobis distance 
3. Two sides pairs searching by faiss 
4. Metrics (ATT, ATC, ATE) estimation depends on your data 

In [4]:
data = data.fillna(method="backfill")

C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:385: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return self.data.fillna(values, method=method, **kwargs)


In [17]:
c_data = copy.deepcopy(data)
test = Matching(bias_estimation=False)
result = test.execute(c_data)

In [18]:
result.full_data[["treat", 'treat_matched']].value_counts()

   treat  treat_matched  count
0      0              0   2744
1      1              0   2557
2      1              1   2507
3      0              1   2192

**ATT** shows the difference in treated group.   
**ATC** shows the difference in untreated group.   
**ATE** shows the weighted average difference between ATT and ATC.   

In [19]:
result.resume

{'ATT': 32.784733192908554,
 'ATC': -35.964141004862235,
 'ATE': -2.0296967008425737}

We can change **metric** and do estimation again.

In [8]:
c_data = copy.deepcopy(data)
test = Matching(metric="atc")
result = test.execute(c_data)

In [9]:
result.resume

{'ATC': -31.600748055935718}

In [10]:
result.full_data

      user_id  signup_month  treat  pre_spends  post_spends   age  gender  \
0           0             0      0       488.0   414.444444  26.0    True   
1           1             8      1       512.5   462.222222  26.0    True   
2           2             7      1       483.0   479.444444  25.0    True   
3           3             0      0       501.5   424.333333  39.0    True   
4           4             1      1       543.0   514.555556  18.0    True   
...       ...           ...    ...         ...          ...   ...     ...   
9995     9995            10      1       538.5   450.444444  42.0    True   
9996     9996             0      0       500.5   430.888889  26.0    True   
9997     9997             3      1       473.0   534.111111  22.0    True   
9998     9998             2      1       495.0   523.222222  67.0    True   
9999     9999             7      1       508.0   475.888889  38.0    True   

      industry  user_id_matched  signup_month_matched  treat_matched  \
0  

Also it is possible to search pairs only in **test group**. This way we have metric "auto" and **ATT** will be estimated. 

In [11]:
c_data = copy.deepcopy(data)
test = Matching(two_sides=False)
result = test.execute(c_data)

In [12]:
result.resume

{'ATC': 173.9774951154092}

In [13]:
result.full_data

      user_id  signup_month  treat  pre_spends  post_spends   age  gender  \
0           0             0      0       488.0   414.444444  26.0    True   
1           1             8      1       512.5   462.222222  26.0    True   
2           2             7      1       483.0   479.444444  25.0    True   
3           3             0      0       501.5   424.333333  39.0    True   
4           4             1      1       543.0   514.555556  18.0    True   
...       ...           ...    ...         ...          ...   ...     ...   
9995     9995            10      1       538.5   450.444444  42.0    True   
9996     9996             0      0       500.5   430.888889  26.0    True   
9997     9997             3      1       473.0   534.111111  22.0    True   
9998     9998             2      1       495.0   523.222222  67.0    True   
9999     9999             7      1       508.0   475.888889  38.0    True   

      industry  user_id_matched  signup_month_matched  treat_matched  \
0  

Finally, we may search pairs in L2 distance. 

In [14]:
c_data = copy.deepcopy(data)
test = Matching(distance="l2", two_sides=False)
result = test.execute(c_data)

In [15]:
result.resume

{'ATC': 202.06190052799212}

In [16]:
result.full_data

      user_id  signup_month  treat  pre_spends  post_spends   age  gender  \
0           0             0      0       488.0   414.444444  26.0    True   
1           1             8      1       512.5   462.222222  26.0    True   
2           2             7      1       483.0   479.444444  25.0    True   
3           3             0      0       501.5   424.333333  39.0    True   
4           4             1      1       543.0   514.555556  18.0    True   
...       ...           ...    ...         ...          ...   ...     ...   
9995     9995            10      1       538.5   450.444444  42.0    True   
9996     9996             0      0       500.5   430.888889  26.0    True   
9997     9997             3      1       473.0   534.111111  22.0    True   
9998     9998             2      1       495.0   523.222222  67.0    True   
9999     9999             7      1       508.0   475.888889  38.0    True   

      industry  user_id_matched  signup_month_matched  treat_matched  \
0  